### SERPAPI 

In [8]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle

from pprint import pprint
from libraries.serpapi import serpapi

In [9]:
# Cargamos los puestos y nuestra API_KEY
today = datetime.now().strftime("%Y-%m-%d")

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()

lista_puestos = pd.read_excel("data/lista_puestos.xlsx", header = None)

lista_puestos.columns = ["jobs"]

lista_puestos

,jobs
0,Desarrollador de software
1,Programador web
2,Consultor tecnológico
3,Ingeniero de sistemas
4,Especialista en seguridad informática
...,...
478,Desarrollador de software científico
479,Técnico de soporte remoto
480,Especialista en redes informáticas
481,Analista de seguridad en la nube


In [ ]:
# Ajustamos España como país de búsqueda
spain = [[".es", "Spain"]]

In [ ]:
%%time

df = pd.DataFrame()

for abr, country in spain:
    
    for q in lista_puestos["jobs"].values:

        try:
            
            for pagination in range(100):
                
                print(f"{q:60}{pagination}")

                q_params = {"q"             : q,
                            "api_key"       : api_key,
                            "location"      : country.title(),
                            "start"         : pagination*10
                            }

                response = serpapi.job_search(**q_params)


                if ("error" in response) or (response.get("jobs_results") == None) or (len(response.get("jobs_results")) < 10):
                    break

                df_response = pd.json_normalize(response["jobs_results"])

                df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

                df_response["country_search"] = country.title()
                
                df = pd.concat(objs = [df, df_response], ignore_index = True)
                
        except:
            
            print(f"Error {q} ***************************************************************")
                 
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True)
df["date_posted"] = datetime.strptime(today, "%Y-%m-%d").date()

df.to_csv(f"data/extraction_{today}_spain.csv", index = False)
df.to_pickle(f"data/extraction_{today}_spain.pkl")

In [3]:
df

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,posted_at,schedule_type,work_from_home,country_search,salary
0,Desarrollador de software Full Stack (JavaScri...,etalentum Selección,"Centelles, Spain",via LinkedIn,Empresa de consolidada trayectoria y en consta...,[{'items': ['Empresa de consolidada trayectori...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"['5 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIHNvZn...,5 hours ago,Full-time,NaN,Spain,NaN
1,Desarrollador Software,Airtificial A&D Engineering,"Seville, Spain",via LinkedIn,En Airtificial A&D Engineering iniciamos un pr...,[{'items': ['En Airtificial A&D Engineering in...,[{'link': 'https://www.google.com/search?sca_e...,NaN,"['7 hours ago', 'Full-time', 'No degree mentio...",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIFNvZnR3YX...,7 hours ago,Full-time,NaN,Spain,NaN
2,Desarrollador/a de software,ALTEN,"San Fernando, Spain",via LinkedIn,¡Únete al equipo Alten y comparte tu experienc...,[{'items': ['¡Únete al equipo Alten y comparte...,"[{'link': 'https://www.alten.fr/', 'text': 'al...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"['3 days ago', 'Full-time', 'No degree mention...",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yL2EgZGUgc2...,3 days ago,Full-time,NaN,Spain,NaN
3,Office Engineer,SGS,Anywhere,via Learn4Good,Company Description We are SGS – the world’s l...,[{'items': ['Company Description We are SGS – ...,"[{'link': 'http://www.sgs.com/', 'text': 'sgs....",https://encrypted-tbn0.gstatic.com/images?q=tb...,"['2 days ago', 'Work from home', 'Full-time', ...",eyJqb2JfdGl0bGUiOiJPZmZpY2UgRW5naW5lZXIiLCJodG...,2 days ago,Full-time,True,Spain,NaN
4,Desarrollador de front-end,Netex Learning,"Oleiros, Spain",via LinkedIn,Somos una empresa del sector e-learning con pr...,[{'items': ['Somos una empresa del sector e-le...,"[{'link': 'http://www.netexlearning.com/', 'te...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"['8 hours ago', 'Full-time', 'No degree mentio...",eyJqb2JfdGl0bGUiOiJEZXNhcnJvbGxhZG9yIGRlIGZyb2...,8 hours ago,Full-time,NaN,Spain,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20515,Soporte Técnico Informático en remoto para por...,Cronoshare,"El Puerto de Santa María, Spain",via BeBee,Necesito un servicio de Informático online en ...,[{'items': ['Necesito un servicio de Informáti...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"['6 days ago', 'Full-time', 'No degree mention...",eyJqb2JfdGl0bGUiOiJTb3BvcnRlIFTDqWNuaWNvIEluZm...,6 days ago,Full-time,NaN,Spain,NaN
20516,SOPORTE TÉCNICO REMOTO (Energías renovables),SEGULA Technologies - Barcelona,"Barcelona, Spain",via Remotify.es,"Descripción:\n\nGrupo Segula, precisa incorpor...","[{'items': ['Descripción:\n\nGrupo Segula, pre...",[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"['Full-time', 'No degree mentioned']",eyJqb2JfdGl0bGUiOiJTT1BPUlRFIFTDiUNOSUNPIFJFTU...,NaN,Full-time,NaN,Spain,NaN
20517,Soporte Técnico Informático en remoto para pc,Cronoshare,"Parla, Spain",via BeBee,"Hola, estoy realizando un curso del cual me gu...","[{'items': ['Hola, estoy realizando un curso d...",[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"['5 days ago', 'Full-time', 'No degree mention...",eyJqb2JfdGl0bGUiOiJTb3BvcnRlIFTDqWNuaWNvIEluZm...,5 days ago,Full-time,NaN,Spain,NaN
20518,Soporte Técnico Informático en remoto para por...,Cronoshare,"Alcobendas, Spain",via BeBee,Necesito un servicio de Informático online en ...,[{'items': ['Necesito un servicio de Informáti...,[{'link': 'https://www.google.com/search?sca_e...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"['6 days ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJTb3BvcnRlIFTDqWNuaWNvIEluZm...,6 days ago,Full-time,NaN,Spain,NaN


In [ ]:
################################################################################################################################